# Decorators

Python decorators allow you to change the behavior of a function without modifying the function tself

You'll use a decorator when you need to change the behavior of a function without modifying the function itself. A few good examples are when you want to add logging, test performance, perform caching, verify permissions, and so on.

You can also use one when you need to run the same code on multiple functions. This avoids you writing duplicating code

A function is an object. Because of that, a function can be assigned to a variable. The function can be accessed from that variable.

In [ ]:
def my_function():

    print('I am a function.')

# Assign the function to a variable without parenthesis. We don't want to execute the function.

description = my_function

In [ ]:
# Accessing the function from the variable I assigned it to.

print(description())



I am a function.
None


'I am a function.'

A function can be nested within another function.

In [ ]:
def outer_function():

    def inner_function():

        print('I came from the inner function.')

    # Executing the inner function inside the outer function.
    inner_function()

In [ ]:
outer_function()

I came from the inner function.


Note that the inner_function is not available outside the outer_function.

 If I try to execute the inner_function outside of the outer_function I receive a NameError exception.

In [ ]:
inner_function()

NameError: ignored

Since a function can be nested inside another function it can also be returned.

In [ ]:
def outer_function():
    '''Assign task to student'''

    task = 'Read Python book chapter 3.'
    def inner_function():
        print(task)
    return inner_function

homework = outer_function()

In [ ]:
homework()

Read Python book chapter 3.


A function can be passed to another function as an argument.

In [ ]:
def friendly_reminder(func):
    '''Reminder for husband'''

    func()
    print('Don\'t forget to bring your wallet!')

def action():

    print('I am going to the store buy you something nice.')

In [ ]:
# Calling the friendly_reminder function with the action function used as an argument.

friendly_reminder(action)


I am going to the store buy you something nice.
Don't forget to bring your wallet!


To create a decorator function in Python, I create an outer function that takes a function as an argument. There is also an inner function that wraps around the decorated function.

Here is the syntax for a basic Python decorator

In [ ]:
def my_decorator_func(func):

    def wrapper_func():
        # Do something before the function.
        func()
        # Do something after the function.
    return wrapper_func

To use a decorator ,you attach it to a function like you see in the code below.

We use a decorator by placing the name of the decorator directly above the function we want to use it on. You prefix the decorator function with an @ symbol.

In [ ]:
@my_decorator_func
def my_func():

    pass

In [ ]:
from datetime import datetime


def log_datetime(func):
    '''Log the date and time of a function'''

    def wrapper():
        print(f'Function: {func.__name__}\nRun on: {datetime.today().strftime("%Y-%m-%d %H:%M:%S")}')
        print(f'{"-"*30}')
        func()
    return wrapper


@log_datetime
def daily_backup():

    print('Daily backup job has finished.')

In [ ]:
daily_backup()

Function: daily_backup
Run on: 2023-10-03 04:58:05
------------------------------
Daily backup job has finished.


Decorators can have arguments passed to them. To add arguments to decorators I add *args and **kwargs to the inner functions.

*args will take an unlimited number of arguments of any type, such as 10, True, or 'Brandon'.
**kwargs will take an unlimited number of keyword arguments, such as count=99, is_authenticated=True, or name='Brandon'

In [ ]:
def my_decorator_func(func):

    def wrapper_func(*args, **kwargs):
        # Do something before the function.
        func(*args, **kwargs)
        # Do something after the function.
    return wrapper_func


@my_decorator_func
def my_func(my_arg):
    '''Example docstring for function'''

    pass

Decorators hide the function they are decorating. If I check the __name__ or __doc__ method we get an unexpected result.

In [ ]:
print(my_func.__name__)
print(my_func.__doc__)

wrapper_func
None


To fix this issue I will use functools. Functools wraps will update the decorator with the decorated functions attributes

In [ ]:
from functools import wraps

def my_decorator_func(func):

    @wraps(func)
    def wrapper_func(*args, **kwargs):
        func(*args, **kwargs)
    return wrapper_func

@my_decorator_func
def my_func(my_args):
    '''Example docstring for function'''

    pass

In [ ]:
print(my_func.__name__)
print(my_func.__doc__)


my_func
Example docstring for function


In [ ]:
from functools import wraps
import tracemalloc
from time import perf_counter


def measure_performance(func):
    '''Measure performance of a function'''

    @wraps(func)
    def wrapper(*args, **kwargs):
        tracemalloc.start()
        start_time = perf_counter()
        func(*args, **kwargs)
        current, peak = tracemalloc.get_traced_memory()
        finish_time = perf_counter()
        print(f'Function: {func.__name__}')
        print(f'Method: {func.__doc__}')
        print(f'Memory usage:\t\t {current / 10**6:.6f} MB \n'
              f'Peak memory usage:\t {peak / 10**6:.6f} MB ')
        print(f'Time elapsed is seconds: {finish_time - start_time:.6f}')
        print(f'{"-"*40}')
        tracemalloc.stop()
    return wrapper


@measure_performance
def make_list1():
    '''Range'''

    my_list = list(range(100000))


@measure_performance
def make_list2():
    '''List comprehension'''

    my_list = [l for l in range(100000)]


@measure_performance
def make_list3():
    '''Append'''

    my_list = []
    for item in range(100000):
        my_list.append(item)


@measure_performance
def make_list4():
    '''Concatenation'''

    my_list = []
    for item in range(100000):
        my_list = my_list + [item]


print(make_list1())
print(make_list2())
print(make_list3())
print(make_list4())

Function: make_list1
Method: Range
Memory usage:		 0.000616 MB 
Peak memory usage:	 3.593484 MB 
Time elapsed is seconds: 0.059018
----------------------------------------
None
Function: make_list2
Method: List comprehension
Memory usage:		 0.003365 MB 
Peak memory usage:	 3.597289 MB 
Time elapsed is seconds: 0.058170
----------------------------------------
None
Function: make_list3
Method: Append
Memory usage:		 0.001096 MB 
Peak memory usage:	 3.594876 MB 
Time elapsed is seconds: 0.072859
----------------------------------------
None
Function: make_list4
Method: Concatenation
Memory usage:		 0.034769 MB 
Peak memory usage:	 4.427879 MB 
Time elapsed is seconds: 18.901216
----------------------------------------
None


I have created a decorator that will measure memory and speed of a function.
We'll use the decorator to test the performance list generation using four methods: range, list comprehension, append, and concatenation.

You can use decorators with classes as well. Let's see how you use decorators with a Python class.

In this example, notice there is no @ character involved. With the __call__ method the decorator is executed when an instance of the class is created.

This class keeps track of the number of times a function to query to an API has been run. Once it reaches the limit the decorator stops the function from executing.

In [ ]:
import requests


class LimitQuery:

    def __init__(self, func):
        self.func = func
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.limit = args[0]
        if self.count < self.limit:
            self.count += 1
            return self.func(*args, **kwargs)
        else:
            print(f'No queries left. All {self.count} queries used.')
            return

@LimitQuery
def get_coin_price(limit):
    '''View the Bitcoin Price Index (BPI)'''

    url = requests.get('https://api.coindesk.com/v1/bpi/currentprice.json')

    if url.status_code == 200:
        text = url.json()
        return f"${float(text['bpi']['USD']['rate_float']):.2f}"


print(get_coin_price(5))
print(get_coin_price(5))
print(get_coin_price(5))
print(get_coin_price(5))
print(get_coin_price(5))
print(get_coin_price(5))

$27605.46
$27605.46
$27605.46
$27605.46
$27605.46
No queries left. All 5 queries used.
None


In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


We've seen Python functions and objects.  Today, we'll breifly cover decorators, which are functions that wrap functions,

```python
@time_wrapper
def my_function(x):
    ...
    
y = my_function(x)
```
```
> 1.1 seconds elapsed.
```

Again, we want to write a function that wraps another function.  We can define this as

In [ ]:
from time import monotonic

def time_wrapper(f):

    def inner_wrapper(*args, **kwargs):
        t0 = monotonic()
        ret = f(*args, **kwargs)
        t1 = monotonic()
        print("{} sec. elapsed".format(t1 - t0))
        return ret

    return inner_wrapper

Here's how we might use our wrapper:

In [ ]:
def generate_random(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random = time_wrapper(generate_random)

x = generate_random(100000)
len(x)

0.001988777003134601 sec. elapsed


100000

one problem with this is that we had to go through the trouble of wrapping the function. Instead, we can just write

In [ ]:
@time_wrapper
def generate_random2(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random2(100000)

0.001884669007267803 sec. elapsed


array([0.98937031, 0.08336035, 0.45845972, ..., 0.53122849, 0.06449548,
       0.91733102])

This is interpreted like what we did above, but is a bit easier to read.

Another problem we may encounter is that we now can't access the docstring we wrote

In [ ]:
help(generate_random2)

Help on function inner_wrapper in module __main__:

inner_wrapper(*args, **kwargs)



One way to solve this is to use the `wraps` decorator from the [functools](https://docs.python.org/3/library/functools.html) package.

In [ ]:
from functools import wraps

def time_wrapper2(f):

    @wraps(f)
    def inner_wrapper(*args, **kwargs):
        t0 = monotonic()
        ret = f(*args, **kwargs)
        t1 = monotonic()
        print("{} sec. elapsed".format(t1 - t0))
        return ret

    return inner_wrapper

@time_wrapper2
def generate_random3(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random3(100000)

0.001709334013867192 sec. elapsed


array([0.72765311, 0.62323703, 0.53931556, ..., 0.41920835, 0.66975116,
       0.30342516])

now, we can access the metadata from `generate_random3`, such as the docstring

In [ ]:
help(generate_random3)

Help on function generate_random3 in module __main__:

generate_random3(n)
    return a random numpy vector



As you might guess, the `@wraps` decorator copies the class metadata from `f` to the `inner_wrapper` function.